In [1]:
import pandas as pd
from numpy.random import seed

from sklearn.model_selection import KFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

baseSeed = 777
from tensorflow import random
from numpy.random import seed
seed(baseSeed)
random.set_seed(baseSeed)

trainFeatures        = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
trainTargetScored    = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
testFeatures         = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
sampleSubmission     = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

trainFeatures['cp_type'] = trainFeatures['cp_type'].map({'trt_cp':0, 'ctl_vehicle':1})
trainFeatures['cp_dose'] = trainFeatures['cp_dose'].map({'D1':0, 'D2':1})
trainFeatures = trainFeatures.drop(columns="sig_id")
trainTargetScored = trainTargetScored.drop(columns="sig_id")

testFeatures['cp_type'] = testFeatures['cp_type'].map({'trt_cp':0, 'ctl_vehicle':1})
testFeatures['cp_dose'] = testFeatures['cp_dose'].map({'D1':0, 'D2':1})
testFeatures = testFeatures.drop(columns="sig_id")

featuresCount = trainFeatures.shape[1]
print("Features count = %d" % featuresCount)

targetsCols  = trainTargetScored.columns
targetsCount = len(targetsCols)
print("Targets count = %d" % targetsCount)


def getModel():
	model = Sequential()
	model.add(Dense(1024, input_dim=featuresCount, activation='relu'))
	model.add(Dropout(0.1))
	model.add(Dense(1024, activation='relu'))
	model.add(Dropout(0.1))
	model.add(Dense(1024, activation='relu'))
	model.add(Dropout(0.1))
	model.add(Dense(1024, activation='relu'))
	model.add(Dropout(0.1))
	model.add(Dense(targetsCount, activation="sigmoid"))
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[])
	return model

epochs = 20
batchSize = 1000
verbose = 1
nSplits = 11


kfold = KFold(n_splits=nSplits, shuffle=True, random_state=baseSeed)
estimator = KerasClassifier(build_fn=getModel, epochs=epochs, batch_size=batchSize, verbose=verbose)
estimator.fit(trainFeatures, trainTargetScored)

predict = estimator.predict_proba(testFeatures)
predictions = sampleSubmission.copy()
predictions.loc[:,targetsCols] = predict
predictions.to_csv('submission.csv', index=False)

Features count = 875
Targets count = 206
Epoch 1/20
24/24 [==============================] - 6s 256ms/step - loss: 0.0939
Epoch 2/20
24/24 [==============================] - 6s 232ms/step - loss: 0.0234
Epoch 3/20
24/24 [==============================] - 6s 244ms/step - loss: 0.0201
Epoch 4/20
24/24 [==============================] - 6s 246ms/step - loss: 0.0189
Epoch 5/20
24/24 [==============================] - 6s 231ms/step - loss: 0.0180
Epoch 6/20
24/24 [==============================] - 6s 242ms/step - loss: 0.0172
Epoch 7/20
24/24 [==============================] - 6s 230ms/step - loss: 0.0165
Epoch 8/20
24/24 [==============================] - 5s 229ms/step - loss: 0.0160
Epoch 9/20
24/24 [==============================] - 6s 231ms/step - loss: 0.0155
Epoch 10/20
24/24 [==============================] - 5s 229ms/step - loss: 0.0151
Epoch 11/20
24/24 [==============================] - 6s 232ms/step - loss: 0.0146
Epoch 12/20
24/24 [==============================] - 6s 243ms/step